In [41]:
import tensorflow as tf
from tensorflow.keras import layers, models
import dlib
import cv2
import numpy as np
import os
from tensorflow.keras.preprocessing import image
import ipywidgets as widgets
from io import BytesIO


In [42]:
# Load the Dlib shape predictor
shape_predictor = dlib.shape_predictor("models/shape_predictor_68_face_landmarks.dat")

# Load your jaundice detection model
jaundice_model = tf.keras.models.load_model("models/jaundice_detector.keras")

In [43]:
def full_face_eye_cropping(img, M = 36, N = 42):
    """
    
    This is the function to crop and save the eye for an input of a full face
    
    Input:
    img = The input image loaded with dlib
    M, N = The landmark region of intrest ((42,46) for left eye and (36,42) for right eye)
    
    Output:
    eye = The cropped image of the input where only the eye is present in the size of (150,150)
    
    """
    
    # Defining Pre-Trained Models
    face_detector = dlib.get_frontal_face_detector()
    landmark_detector = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    
    # Detecting the faces in the input
    faces = face_detector(img, 1)
    face = faces[0]
    
    # Detecting landmarks in the image
    base_img = img.copy()
    landmarks_tuple = []
    landmarks = landmark_detector(img,face)

    for i in range(0, 68):
        x = landmarks.part(i).x
        y = landmarks.part(i).y

        landmarks_tuple.append((x, y))
            
        cv2.circle(base_img, (x, y), 2, (255, 255, 255), -1)
    
    # Creating a route between landmarks
    routes = [i for i in range(M,N)] + [M]
    route_coordinates = []
    base_img = img.copy()

    # Filling route_coordinates
    for i in range(0, len(routes) - 1):
        source_point = routes[i]
        target_point = routes[i+1]
    
        source_coordinate = landmarks_tuple[source_point]
        target_coordinate = landmarks_tuple[target_point]
    
        route_coordinates.append(source_coordinate)
    
        cv2.line(base_img, source_coordinate, target_coordinate, (255, 255, 255), 2)
        
    # Getting the eye bounding box using route_coordinates
    x_coords = [coord[0] for coord in route_coordinates]
    y_coords = [coord[1] for coord in route_coordinates]
    x_min, x_max = min(x_coords), max(x_coords)
    y_min, y_max = min(y_coords), max(y_coords)

    # Add padding
    padding = 10
    x_min = max(0, x_min - padding)
    y_min = max(0, y_min - padding)
    x_max = min(img.shape[1], x_max + padding)
    y_max = min(img.shape[0], y_max + padding)

    # Crop and resize
    eye_region = img[y_min:y_max, x_min:x_max]
    eye = cv2.resize(eye_region, (150, 150))
    
    return eye

In [44]:
class PreprocessingLayer(tf.keras.layers.Layer):
    def __init__(self, target_size=(150, 150)):
        super(PreprocessingLayer, self).__init__()
        self.target_size = target_size

    def call(self, inputs):
        # Preprocessing step: resizing the image
        x = tf.image.resize(inputs, self.target_size)  # Resize input image to the target size
        return x

    def compute_output_shape(self, input_shape):
        # The output shape is determined by the target size of the preprocessing layer
        return (input_shape[0], self.target_size[0], self.target_size[1], input_shape[3])

In [45]:
# Build the integrated model
inputs = tf.keras.Input(shape=(None, None, 3))  # Adjust input shape as needed
x = PreprocessingLayer()(inputs)
outputs = jaundice_model(x)
integrated_model = tf.keras.Model(inputs, outputs)
integrated_model.summary()

Model: "functional_38"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, None, None, 3)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ preprocessing_layer_3           │ (None, 150, 150, 3)    │             0 │
│ (PreprocessingLayer)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_1 (Sequential)       │ (None, 1)              │     9,494,561 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,494,561 (36.22 MB)

 Trainable params: 9,494,561 (36.22 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# # Save the integrated model
# integrated_model.save("integrated_model.keras")

In [ ]:
# # Convert the model to TFLite
# converter = tf.lite.TFLiteConverter.from_keras_model(integrated_model)
# tflite_model = converter.convert()

# # Save the converted model to a file
# with open('final_model.tflite', 'wb') as f:
#     f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\muham\AppData\Local\Temp\tmpg55k11bp\assets


INFO:tensorflow:Assets written to: C:\Users\muham\AppData\Local\Temp\tmpg55k11bp\assets


Saved artifact at 'C:\Users\muham\AppData\Local\Temp\tmpg55k11bp'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, None, None, 3), dtype=tf.float32, name='keras_tensor_187')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  2773950786976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2773950009184: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2773948941232: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2773951302112: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2773951313200: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2773951310032: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2773951314960: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2773951483216: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2773951485328: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2773951488144: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [ ]:
# import pickle

# # Save the model object as a .pkl file
# with open('final_model.pkl', 'wb') as f:
#     pickle.dump(integrated_model, f)


In [ ]:
# import json

# # Save the architecture (structure) to a .json file
# model_json = integrated_model.to_json()  # Get the architecture as a JSON string

# # Write the JSON string to a file
# with open('final_model.json', 'w') as json_file:
#     json.dump(model_json, json_file)

In [47]:
# Compile the model before evaluation (if not already compiled)
integrated_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [48]:
# Define the directories for jaundiced and normal eye images
BASE_DIR = "./Data/"
jaundiced_dir = os.path.join(BASE_DIR, "Jaundiced Eyes/")
normal_dir = os.path.join(BASE_DIR, "Normal Eyes/")

# Load images from directories
jaundiced_images = [os.path.join(jaundiced_dir, fname) for fname in os.listdir(jaundiced_dir)]
normal_images = [os.path.join(normal_dir, fname) for fname in os.listdir(normal_dir)]

In [49]:
# Assign labels (1 for jaundiced, 0 for normal)
test_images = jaundiced_images + normal_images
test_labels = [1] * len(jaundiced_images) + [0] * len(normal_images)

In [50]:
# Preprocess test images: Resize and preprocess as you did for training
def preprocess_eye(img_path):
    img = image.load_img(img_path, target_size=(150, 150))  # Resize to match input shape
    img_array = image.img_to_array(img)
    img_array = img_array / 255.0  # Normalize if necessary (depends on your training preprocessing)
    return img_array

test_images_preprocessed = []
for img_path in test_images:
    preprocessed_image = preprocess_eye(img_path)
    test_images_preprocessed.append(preprocessed_image)

# Convert the images to a numpy array
test_images_preprocessed = np.array(test_images_preprocessed)

# Evaluate the model
loss, accuracy = integrated_model.evaluate(test_images_preprocessed, np.array(test_labels))

# Print the accuracy
print(f"Test accuracy: {accuracy * 100:.2f}%")

15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.6884 - loss: 0.6198
Test accuracy: 37.50%


In [51]:
uploader = widgets.FileUpload(accept="image/*", multiple=True)
display(uploader)
out = widgets.Output()
display(out)

rescale_layer = tf.keras.layers.Rescaling(1./255)

def file_predict(filename, file, out):
    """ A function for creating the prediction and printing the output."""
    image = tf.keras.utils.load_img(file, target_size=(150, 150))
    image = tf.keras.utils.img_to_array(image)
    image = rescale_layer(image)
    image = np.expand_dims(image, axis=0)

    prediction = integrated_model.predict(image, verbose=0)[0][0]
    print(prediction)

    with out:
        if prediction > 0.5:
            print(filename + " is a jaundiced eye")
        else:
            print(filename + " is a normal eye")


def on_upload_change(change):
    """ A function for geting files from the widget and running the prediction."""

    items = change.new
    for item in items: # Loop if there is more than one file uploaded
        file_jpgdata = BytesIO(item.content)
        file_predict(item.name, file_jpgdata, out)
uploader.observe(on_upload_change, names='value')

FileUpload(value=(), accept='image/*', description='Upload', multiple=True)

Output()